In [1]:
!pip install flask
!pip install pyngrok
!pip install transformers
!pip install torch

In [2]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from threading import Thread
from transformers import pipeline

!ngrok authtoken 2m9Q010CrnXZutxYI6IvFup03jW_6xatV1EjsqTequfPGdJ7x


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
classifier = pipeline(
    "text-classification",
    model="IsmaelMousa/bert-finetuned-emotion",
    return_all_scores=True
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [30]:
app = Flask(__name__)

@app.route('/classify', methods=['POST'])
def classify_text():
    data = request.get_json()
    if not data or 'text' not in data:
        return jsonify({'error': "Please provide text to classify in JSON format with a 'text' key."}), 400
    text = data['text']
    result = classifier(text)[0][0]['label']
    return jsonify(result)

public_url = ngrok.connect(5002).public_url
print('Public URL:', public_url)

def run_app():
    app.run(port=5005)

thread = Thread(target=run_app)
thread.start()


Public URL: https://956d-35-237-171-53.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


In [31]:
import requests

url = public_url+'/classify'  # Replace with your public URL
data = {'text': 'I am crying, my dog died'}

response = requests.post(url, json=data)
print(response.json())


INFO:werkzeug:127.0.0.1 - - [20/Sep/2024 11:32:06] "POST /classify HTTP/1.1" 200 -


sadness


In [17]:
# Kill the ngrok tunnel
ngrok.kill()

In [28]:
while True:pass


INFO:werkzeug:127.0.0.1 - - [20/Sep/2024 11:30:31] "POST /classify HTTP/1.1" 200 -


KeyboardInterrupt: 